# **Homework 5 - USA Airport Flight Analysis**

*Group#12*

- **Marco Zimmatore** - [zimmatore.1947442@studenti.uniroma1.it](mailto:zimmatore.1947442@studenti.uniroma1.it)
- **Davide Vitale** - [vitale.1794386@studenti.uniroma1.it](mailto:vitale.1794386@studenti.uniroma1.it)
- **Darkhan Maksutov** - [maksutov.2113209@studenti.uniroma1.it](mailto:maksutov.2113209@studenti.uniroma1.it)
- **Riccardo Soleo** - [soleo.1911063@studenti.uniroma1.it](mailto:soleo.1911063@studenti.uniroma1.it)

___

In [1]:
import functions
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from IPython.display import display, Markdown
import warnings
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Suppress all warnings
warnings.filterwarnings("ignore")
dataset_path = "../archive/Airports2.csv"       

In [46]:
import importlib
importlib.reload(functions)

<module 'functions' from 'c:\\Users\\Marco\\Desktop\\Magistrale\\ADM\\ADM-HW5\\ADM-HW5\\functions.py'>

In [3]:
df = pd.read_csv(dataset_path)
df.head()

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.140999,-96.670799,NaN,NaN
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103,1990-11-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103,1990-12-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103,1990-10-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156,1990-02-01,147300,76034,42.374199,-122.873001,44.254101,-121.150002


> We wanna build a graph with edges for each route, where each node represents an airport and eac edge is the existent route between two airports, with all the infos on that route. For this reason, we decide to remove all the rows with *NA* values in them, because each information is crucial for the next exercises computations.

In [4]:
df = df.dropna()
df = df[df['Origin_airport'] != df['Destination_airport']]
df.head()

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103,1990-11-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103,1990-12-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103,1990-10-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156,1990-02-01,147300,76034,42.374199,-122.873001,44.254101,-121.150002
5,MFR,RDM,"Medford, OR","Bend, OR",11,18,1,156,1990-03-01,147300,76034,42.374199,-122.873001,44.254101,-121.150002


## **Flight Network Analysis (Q1)**

##### **Function** `create_airport_graph`

In [5]:
graph = functions.create_airport_graph(df)

1. Implement a function `analyze_graph_features(flight_network)` that takes the flight network as input and computes the following:

    - Count the number of airports (`nodes`) and flights (`edges`) in the graph.

    - Compute the density of the graph using the formula: $ Density = \frac{2\times E}{N(N − 1)}$

    - Calculate both `in-degree` and `out-degree` for each airport and visualize them using histograms.

    - Identify airports with degrees higher than the 90th percentile and list them as "`hubs`".
    
    - Determine if the graph is sparse or dense based on its density.



In [6]:
def analyze_graph_features(flight_network):

    number_of_nodes = 0
    number_of_edges = len(flight_network.edges())
    dict_degrees_edges = dict()

    for node in flight_network.nodes:
        number_of_nodes = number_of_nodes + 1

        in_edges = 0
        out_edges = 0

        for _, _, attr in flight_network.edges(node, data = True):
            out_edges+=1

        for _, _, attr in flight_network.in_edges(node, data = True):
            in_edges+=1
            

        dict_degrees_edges[node] = [in_edges, out_edges]

    graph_density = (2 * number_of_edges) / (number_of_nodes* (number_of_nodes -1))
    

    in_degrees = [edge_degree[0] for edge_degree in dict_degrees_edges.values()]
    out_degrees = [edge_degree[1] for edge_degree in dict_degrees_edges.values()]
    


    # Create a subplot with 1 row, 2 columns
    fig = make_subplots(rows=1, cols=2, subplot_titles=('In-degree Histogram', 'Out-degree Histogram'))

    # Add in-degree histogram in the first subplot (left)
    fig.add_trace(
        go.Histogram(x=in_degrees, nbinsx=20, name='In-degree', marker=dict(color='steelblue')),
        row=1, col=1
    )

    # Add out-degree histogram in the second subplot (right)
    fig.add_trace(
        go.Histogram(x=out_degrees, nbinsx=20, name='Out-degree', marker=dict(color='darkorange')),
        row=1, col=2
    )

    # Update layout for better aesthetics
    fig.update_layout(
        title="In-degree vs Out-degree Histograms",
        xaxis_title="Degree",
        yaxis_title="Frequency",
        showlegend=True,
        height=500,  # adjust the height of the figure
        width=1000   # adjust the width of the figure
    )



    # Calculate 90th percentile for in-degrees and out-degrees

    # Firstly we build a dictionary to compute the total degree value for each airport(node)
    dict_degrees = dict()

    for node, degrees in dict_degrees_edges.items():
        dict_degrees[node] = degrees[0] + degrees[1]
    

    # wE use np.precentile to obtaine the percentile from the dictionary of the degrees for each node 
    degree_percentile = np.percentile(list(dict_degrees.values()), 90)
    
    # Identify airports that are "hubs" (in-degree or out-degree greater than 90th percentile)
    hubs = []
    
    # Check for hubs
    for node,degree in dict_degrees.items():
        if degree > degree_percentile:
            hubs.append((node, degree))

    threshold = 0.5
    # We check if the Graph is dense or sparse
    if graph_density > threshold:
        is_sparse = False
    else:
        is_sparse = True

    return number_of_nodes, number_of_edges, fig, hubs, is_sparse

2. Write a function `summarize_graph_features(flight_network)` that generates a detailed report of the graph's features. A summary report needs to include:

    - The number of nodes and edges.
    
    - The graph density.
    
    - Degree distribution plots for in-degree and out-degree.
    
    - A table of identified hubs.



In [7]:
def summarize_graph_features(flight_network):
    # Analyze graph features
    number_of_nodes, number_of_edges, degree_histogram, hubs, is_sparse = analyze_graph_features(flight_network)

    # Create a textual summary
    density_description = "dense" if not is_sparse else "sparse"
    summary_table = f"""
| Metric                  | Value                      |
|-------------------------|----------------------------|
| **Number of Airports**      | {number_of_nodes}          |
| **Number of Flights**       | {number_of_edges}          |
| **Graph Density**           | {'{:.4f}'.format((2 * number_of_edges) / (number_of_nodes * (number_of_nodes - 1)))}|
| **Graph Classification**    | {density_description.capitalize()} |
"""

    row_labels = "| Hubs (Airports)          | " + " | ".join([hub[0] for hub in hubs]) + " |\n"
    separator_row = "|-----------------| " + " | ".join(["---"] * len(hubs)) + " |\n"
    # Create the degree row
    degree_row = "| **Degrees**          | " + " | ".join([str(hub[1]) for hub in hubs]) + " |\n"

    # Combine rows into the Markdown table
    hubs_table = row_labels + separator_row + degree_row

    display(Markdown("## **Graph Features Summary**"))

    # Display summary
    display(Markdown(summary_table))

    display(Markdown("### **Identified Hubs**"))
    # Display the hubs table
    display(Markdown(hubs_table))

    # Display the degree distribution histogram
    display(Markdown("### **Degree Distribution**"))
    degree_histogram.show()



3. Now let's dive deeper into the analysis of the dataset. Do the following:
    
    - Compute total passenger flow between origin and destination cities.

    - Identify and visualize the busiest routes by passenger traffic.

    - Calculate the average passengers per flight for each route and highlight under/over-utilized connections.

    - Create an interactive map visualizing the geographic spread of the flight network.

In [8]:
df_sorted_passengers, fig, df_most_traffic, df_least_traffic = functions.analysis_traffic_passengers(df) 
fig.show()


In [9]:
functions.create_interactive_map(df)

Saved map as 'flight_network_map.html'.


4. Once you have created and tested the previous functions, the results should be presented in a tidy way. Your summary report should contain:

- The number of nodes and edges.

-    The graph density.

-    Degree distribution plots for in-degree and out-degree.

-    A table of identified hubs.

-    Top routes by passenger flow (table and bar chart).

-    Top routes by passenger efficiency (table and bar chart).

-    An interactive map showing flight routes.


In [10]:
functions.generate_report(df, graph)

## **Graph Features Summary**


| Metric                  | Value                      |
|-------------------------|----------------------------|
| **Number of Airports (Nodes)**      | 483          |
| **Number of Flights (Edges)**       | 34531          |
| **Graph Density**           | 0.2967|
| **Graph Classification**    | Sparse |


### **Identified Hubs**

| Hubs (Airports)          | SEA | SFO | LAX | FLL | PHX | TUS | DFW | SLC | LAS | ICT | OKC | IAH | ELP | TUL | OMA | RFD | MKE | LIT | SHV | MCI | SAT | MSP | ORD | STL | BNA | MEM | IND | CLE | DTW | DAY | CVG | CMH | PIT | BOS | ATL | MDW | PHL | EWR | CLT | JFK | YIP | MCO | IAD | MSY | RDU | BWI | TYS | MIA | DAL |
|-----------------| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| **Degrees**          | 390 | 366 | 435 | 374 | 474 | 415 | 494 | 457 | 461 | 384 | 453 | 437 | 389 | 365 | 383 | 381 | 451 | 402 | 401 | 496 | 382 | 566 | 512 | 499 | 428 | 523 | 496 | 427 | 455 | 373 | 436 | 395 | 433 | 385 | 509 | 406 | 421 | 422 | 410 | 404 | 541 | 409 | 467 | 385 | 407 | 439 | 382 | 479 | 382 |


### **Degree Distribution**

...

Origin_airport,Destination_airport,Average_Passengers
ABE,ACT,0.0
IDA,SGU,0.0
ILG,ABE,0.0
ILG,ADS,0.0
ILG,BDL,0.0
ILG,BIF,0.0
ILG,BNA,0.0
ILG,BRO,0.0
ILG,BUF,0.0
ILG,CAK,0.0


### **Over-Utilized Routes**

Origin_airport,Destination_airport,Average_Passengers
DAL,HOU,21828.4
HOU,DAL,21686.0
LGA,DCA,15371.6
DCA,LGA,14628.6
HNL,OGG,14043.8
BOS,LGA,13865.1
LGA,BOS,13674.9
OGG,HNL,13490.9
OAK,JFK,12615.6
HOU,MSY,11942.8


### **Top Routes by Passenger Efficiency**

Origin_airport,Destination_airport,Passenger_Efficiency
HNL,OGG,140.4
OGG,HNL,134.9
DAL,HOU,91.3
HOU,DAL,90.7
BOS,LGA,74.9
LGA,BOS,73.9
LGA,DCA,71.8
DCA,LGA,68.4
IAH,EFD,49.7
DAL,AUS,46.5


____
After completing the analysis, answer the following questions:



- Is the graph sparse or dense?


> Since the density of the graph is quite near 0, we can assume that the graph is **sparse**.

-  What patterns do you observe in the degree distribution?


> Both Histograms of *out_degree* and *in_degree* of each edge assume similar distributions: most of the nodes tend to be have 20 or less number of edges, another slice of nodes have in and out degrees between 20 and 100.  

- Which airports are identified as hubs, and why?


| Hubs (Airports)          | SEA | SFO | LAX | FLL | PHX | TUS | DFW | SLC | LAS | ICT | OKC | IAH | ELP | TUL | OMA | RFD | MKE | LIT | SHV | MCI | SAT | MSP | ORD | STL | BNA | MEM | IND | CLE | DTW | DAY | CVG | CMH | PIT | BOS | ATL | MDW | PHL | EWR | CLT | JFK | YIP | MCO | IAD | MSY | RDU | BWI | TYS | MIA | DAL |
|-----------------| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| **Degrees**          | 392 | 368 | 437 | 376 | 476 | 417 | 496 | 459 | 463 | 386 | 455 | 439 | 391 | 367 | 383 | 383 | 453 | 404 | 403 | 498 | 384 | 568 | 514 | 501 | 430 | 525 | 498 | 429 | 457 | 375 | 438 | 397 | 435 | 387 | 511 | 408 | 423 | 424 | 412 | 406 | 543 | 411 | 469 | 387 | 409 | 441 | 384 | 481 | 384 |

<br>

> These airports are listed as '*hubs*' because their degree os higher than 90% of the degree of all the nodes

- What are the busiest routes in terms of passenger traffic?


<style type="text/css">
</style>
<table id="T_c0f38">
  <thead>
    <tr>
      <th id="T_c0f38_level0_col0" class="col_heading level0 col0" >Origin_airport</th>
      <th id="T_c0f38_level0_col1" class="col_heading level0 col1" >Destination_airport</th>
      <th id="T_c0f38_level0_col2" class="col_heading level0 col2" >Total_Passengers</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td id="T_c0f38_row0_col0" class="data row0 col0" >OGG</td>
      <td id="T_c0f38_row0_col1" class="data row0 col1" >HNL</td>
      <td id="T_c0f38_row0_col2" class="data row0 col2" >32364612</td>
    </tr>
    <tr>
      <td id="T_c0f38_row1_col0" class="data row1 col0" >HNL</td>
      <td id="T_c0f38_row1_col1" class="data row1 col1" >OGG</td>
      <td id="T_c0f38_row1_col2" class="data row1 col2" >29744742</td>
    </tr>
    <tr>
      <td id="T_c0f38_row2_col0" class="data row2 col0" >LAX</td>
      <td id="T_c0f38_row2_col1" class="data row2 col1" >HNL</td>
      <td id="T_c0f38_row2_col2" class="data row2 col2" >28964154</td>
    </tr>
    <tr>
      <td id="T_c0f38_row3_col0" class="data row3 col0" >HNL</td>
      <td id="T_c0f38_row3_col1" class="data row3 col1" >LAX</td>
      <td id="T_c0f38_row3_col2" class="data row3 col2" >28632161</td>
    </tr>
    <tr>
      <td id="T_c0f38_row4_col0" class="data row4 col0" >LAS</td>
      <td id="T_c0f38_row4_col1" class="data row4 col1" >LAX</td>
      <td id="T_c0f38_row4_col2" class="data row4 col2" >26333721</td>
    </tr>
    <tr>
      <td id="T_c0f38_row5_col0" class="data row5 col0" >LAX</td>
      <td id="T_c0f38_row5_col1" class="data row5 col1" >LAS</td>
      <td id="T_c0f38_row5_col2" class="data row5 col2" >26177809</td>
    </tr>
    <tr>
      <td id="T_c0f38_row6_col0" class="data row6 col0" >LAX</td>
      <td id="T_c0f38_row6_col1" class="data row6 col1" >SFO</td>
      <td id="T_c0f38_row6_col2" class="data row6 col2" >25661782</td>
    </tr>
    <tr>
      <td id="T_c0f38_row7_col0" class="data row7 col0" >SFO</td>
      <td id="T_c0f38_row7_col1" class="data row7 col1" >LAX</td>
      <td id="T_c0f38_row7_col2" class="data row7 col2" >25458207</td>
    </tr>
    <tr>
      <td id="T_c0f38_row8_col0" class="data row8 col0" >ATL</td>
      <td id="T_c0f38_row8_col1" class="data row8 col1" >MCO</td>
      <td id="T_c0f38_row8_col2" class="data row8 col2" >23483751</td>
    </tr>
    <tr>
      <td id="T_c0f38_row9_col0" class="data row9 col0" >ORD</td>
      <td id="T_c0f38_row9_col1" class="data row9 col1" >LAX</td>
      <td id="T_c0f38_row9_col2" class="data row9 col2" >22979359</td>
    </tr>
  </tbody>
</table>
<br>

> These routes represent the 10 highest traffic connections, highlighting the demand for travel between major airports, especially in regions like California and Hawaii.


- Which routes are under/over-utilized?


##### **Under-Utilized Routes**

<style type="text/css">
</style>
<table id="T_c15c7">
  <thead>
    <tr>
      <th id="T_c15c7_level0_col0" class="col_heading level0 col0" >Origin_airport</th>
      <th id="T_c15c7_level0_col1" class="col_heading level0 col1" >Destination_airport</th>
      <th id="T_c15c7_level0_col2" class="col_heading level0 col2" >Average_Passengers</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td id="T_c15c7_row0_col0" class="data row0 col0" >ABE</td>
      <td id="T_c15c7_row0_col1" class="data row0 col1" >ACT</td>
      <td id="T_c15c7_row0_col2" class="data row0 col2" >0.0</td>
    </tr>
    <tr>
      <td id="T_c15c7_row1_col0" class="data row1 col0" >IDA</td>
      <td id="T_c15c7_row1_col1" class="data row1 col1" >SGU</td>
      <td id="T_c15c7_row1_col2" class="data row1 col2" >0.0</td>
    </tr>
    <tr>
      <td id="T_c15c7_row2_col0" class="data row2 col0" >ILG</td>
      <td id="T_c15c7_row2_col1" class="data row2 col1" >ABE</td>
      <td id="T_c15c7_row2_col2" class="data row2 col2" >0.0</td>
    </tr>
    <tr>
      <td id="T_c15c7_row3_col0" class="data row3 col0" >ILG</td>
      <td id="T_c15c7_row3_col1" class="data row3 col1" >ADS</td>
      <td id="T_c15c7_row3_col2" class="data row3 col2" >0.0</td>
    </tr>
    <tr>
      <td id="T_c15c7_row4_col0" class="data row4 col0" >ILG</td>
      <td id="T_c15c7_row4_col1" class="data row4 col1" >BDL</td>
      <td id="T_c15c7_row4_col2" class="data row4 col2" >0.0</td>
    </tr>
    <tr>
      <td id="T_c15c7_row5_col0" class="data row5 col0" >ILG</td>
      <td id="T_c15c7_row5_col1" class="data row5 col1" >BIF</td>
      <td id="T_c15c7_row5_col2" class="data row5 col2" >0.0</td>
    </tr>
    <tr>
      <td id="T_c15c7_row6_col0" class="data row6 col0" >ILG</td>
      <td id="T_c15c7_row6_col1" class="data row6 col1" >BNA</td>
      <td id="T_c15c7_row6_col2" class="data row6 col2" >0.0</td>
    </tr>
    <tr>
      <td id="T_c15c7_row7_col0" class="data row7 col0" >ILG</td>
      <td id="T_c15c7_row7_col1" class="data row7 col1" >BRO</td>
      <td id="T_c15c7_row7_col2" class="data row7 col2" >0.0</td>
    </tr>
    <tr>
      <td id="T_c15c7_row8_col0" class="data row8 col0" >ILG</td>
      <td id="T_c15c7_row8_col1" class="data row8 col1" >BUF</td>
      <td id="T_c15c7_row8_col2" class="data row8 col2" >0.0</td>
    </tr>
    <tr>
      <td id="T_c15c7_row9_col0" class="data row9 col0" >ILG</td>
      <td id="T_c15c7_row9_col1" class="data row9 col1" >CAK</td>
      <td id="T_c15c7_row9_col2" class="data row9 col2" >0.0</td>
    </tr>
  </tbody>
</table>

<br>

> These routes are under-utilized considering the assumption that we have not considered routes that have no *Distance*, with no passengers recorded on average. This may indicate low demand or possibly infrequent flights, which could suggest an opportunity for route optimization or better marketing strategies to increase passenger traffic.


##### **Over-Utilized Routes**

<style type="text/css">
</style>
<table id="T_fc661">
  <thead>
    <tr>
      <th id="T_fc661_level0_col0" class="col_heading level0 col0" >Origin_airport</th>
      <th id="T_fc661_level0_col1" class="col_heading level0 col1" >Destination_airport</th>
      <th id="T_fc661_level0_col2" class="col_heading level0 col2" >Average_Passengers</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td id="T_fc661_row0_col0" class="data row0 col0" >DAL</td>
      <td id="T_fc661_row0_col1" class="data row0 col1" >HOU</td>
      <td id="T_fc661_row0_col2" class="data row0 col2" >21828.4</td>
    </tr>
    <tr>
      <td id="T_fc661_row1_col0" class="data row1 col0" >HOU</td>
      <td id="T_fc661_row1_col1" class="data row1 col1" >DAL</td>
      <td id="T_fc661_row1_col2" class="data row1 col2" >21686.0</td>
    </tr>
    <tr>
      <td id="T_fc661_row2_col0" class="data row2 col0" >LGA</td>
      <td id="T_fc661_row2_col1" class="data row2 col1" >DCA</td>
      <td id="T_fc661_row2_col2" class="data row2 col2" >15371.6</td>
    </tr>
    <tr>
      <td id="T_fc661_row3_col0" class="data row3 col0" >DCA</td>
      <td id="T_fc661_row3_col1" class="data row3 col1" >LGA</td>
      <td id="T_fc661_row3_col2" class="data row3 col2" >14628.6</td>
    </tr>
    <tr>
      <td id="T_fc661_row4_col0" class="data row4 col0" >HNL</td>
      <td id="T_fc661_row4_col1" class="data row4 col1" >OGG</td>
      <td id="T_fc661_row4_col2" class="data row4 col2" >14043.8</td>
    </tr>
    <tr>
      <td id="T_fc661_row5_col0" class="data row5 col0" >BOS</td>
      <td id="T_fc661_row5_col1" class="data row5 col1" >LGA</td>
      <td id="T_fc661_row5_col2" class="data row5 col2" >13865.1</td>
    </tr>
    <tr>
      <td id="T_fc661_row6_col0" class="data row6 col0" >LGA</td>
      <td id="T_fc661_row6_col1" class="data row6 col1" >BOS</td>
      <td id="T_fc661_row6_col2" class="data row6 col2" >13674.9</td>
    </tr>
    <tr>
      <td id="T_fc661_row7_col0" class="data row7 col0" >OGG</td>
      <td id="T_fc661_row7_col1" class="data row7 col1" >HNL</td>
      <td id="T_fc661_row7_col2" class="data row7 col2" >13490.9</td>
    </tr>
    <tr>
      <td id="T_fc661_row8_col0" class="data row8 col0" >OAK</td>
      <td id="T_fc661_row8_col1" class="data row8 col1" >JFK</td>
      <td id="T_fc661_row8_col2" class="data row8 col2" >12615.6</td>
    </tr>
    <tr>
      <td id="T_fc661_row9_col0" class="data row9 col0" >HOU</td>
      <td id="T_fc661_row9_col1" class="data row9 col1" >MSY</td>
      <td id="T_fc661_row9_col2" class="data row9 col2" >11942.8</td>
    </tr>
  </tbody>
</table>
<br>

> These routes are characterized by high passenger traffic, indicating strong demand, such as *Dallas to Houston* and *Houston to Dallas* that the ones with the highest average.
____

## **Nodes' Contribution (Q2)**

1. Implement a function `analyze_centrality(flight_network, airport)` that computes the following centrality measures for a given airport:


    - *Betweenness* *centrality*: Measures how often a node appears on the shortest paths between other nodes.
    
    - *Closeness* *centrality*: Measures how easily a node can access all other nodes in the network.
    
    - *Degree* *centrality*: Simply counts the number of direct connections to the node.
    
    - *PageRank*: Computes the "importance" of a node based on incoming connections and their weights.


2.  Write a function `compare_centralities(flight_network)` to:

    - Compute and compare centrality values for all nodes in the graph.
    
    - Plot centrality distributions (histograms for each centrality measure).
    
    - Return the top 5 airports for each centrality measure.



3. Ask LLM (eg. ChatGPT) to suggest alternative centrality measures that might be relevant to this task. How can you check that the results given by the LLM are trustable?

4. Implement one of these measures suggested by the LLM, compare its results to the centralities you've already computed, and analyze whether it adds any new insights.

## **Finding Best Routes (Q3)**

- In this task, you need to implement a function that, given an origin and destination city, determines the best possible route between them. To simplify, the focus will be limited to flights operating on a specific day.

**Note**: Each city may have multiple airports; in such cases, the function should calculate the best route for every possible airport pair between the two cities. For example, if city A has airports $a_1$ , $a_2$ and city B has $b_1$ , $b_2$ , the function should compute the best routes for $a_1 → b_1$ , $a_1 → b_2$ , $a_2 → b_1$ and $a_2 → b_2$ . If it’s not possible to travel from one airport in the origin city to another airport in the destination city on that date, you must report it as well.

The function takes the following inputs:

1. Flights network

2. Origin city name

3. Destination city name

4. Considered Date (in yyyy-mm-dd format)

The function output:

1. A table with three columns: 'Origin_city_airport', 'Destination_city_airport', and the 'Best_route'.

**Note**: In the "Best_route" column, we expect a list of airport names connected by → , showing the order in which they are to be visited during the optimal route. If no such route exists, the entry should display "No route found."
____

> We choose the date with the most flights in the dataset

In [40]:
Date = df.groupby('Fly_date')['Origin_airport'].count().idxmax()
Date

'2007-12-01'

> After analyzing our `distances_dictionary` from *Seattle*, we choose one of the routes with the highest `best_distance` in order to highlight the sequence of flights that brings to the destination. 

In [48]:
Origin_city = "Seattle"
Destination_city = "Bloomington"

table = functions.compute_best_route(graph, Origin_city, Destination_city, Date)
# To hide the index of the row (only row of the dataframe)
table.style.hide(axis="index")

Origin_city_airport,Destination_city_airport,Best_route
Seattle,Bloomington,BFI->JFK->TCL->BDL->PWM->AVP->BMI


## **Airline Network Partitioning (Q4)**

- In graph theory, this task is known as a graph disconnection problem. Your goal is to write a function that removes the minimum number of flights between airports to separate the original flight network into two disconnected subgraphs.

## **Finding and Extracting Communities (Q5)**

1. In this task, you are asked to analyze the graph and identify the communities based on the flight network provided. For the airline, the primary focus is on the cities, so your communities should reflect the connectivity between cities through the flights that link them.

2. Ask a LLM (ChatGPT, Claude AI, Gemini, Perplexity, etc.) to suggest an alternative algorithm for extracting communities and explain the steps required to implement it. Then, implement this algorithm and compare its results with the current method you've chosen. Discuss the differences in the outcomes and analyze which approach you think is better, providing reasons for your choice.

## **Bonus Question - Connected Components on MapReduce**

1. In this task, you are required to use PySpark and the MapReduce paradigm to identify the connected components in a flight network graph. The focus should be on airports rather than cities. As you know, a connected component refers to a group of airports where every pair of airports within the group is connected either directly or indirectly.

2. Compare the execution time and the results of your implementation with those of the GraphFrames package for identifying connected components. If there is any difference in the results, provide an explanation for why that might occur.

## **Algorithmic Question (AQ)**

Arya needs to travel between cities using a network of flights. Each flight has a fixed cost (in euros), and she wants to find the cheapest possible way to travel from her starting city to her destination city. However, there are some constraints on the journey:

- Arya can make at most `k` stops during her trip (this means up to `k+1` flights).
    
- If no valid route exists within these constraints, the result should be `-1`.

Given a graph of cities connected by flights, your job is to find the minimum cost for Arya to travel between two specified cities (`src` to `dst`) while following the constraints.

**a)** Write a pseudocode that describes the algorithm to find the cheapest route with at most k stops.

**b)** Implement the algorithm in Python and simulate the given test cases.

**c)** Analyze the algorithm's efficiency. Provide its time complexity and space complexity, and explain whether it is efficient for large graphs (e.g., `n > 100`).

**d)** Optimize the algorithm to handle larger graphs. Provide an updated pseudocode and analyze the computational complexity of your optimization.

**e)** Ask LLM (e.g., ChatGPT) for an optimized version of your algorithm. Compare its solution to yours in terms of performance, time complexity, and correctness.